In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# Make sure that caffe is on the python path:
caffe_root = '../../'  # this file is expected to be in {caffe_root}/examples
import sys
sys.path.insert(0, caffe_root + 'python')
#print sys.path

import caffe

../../python/caffe/pycaffe.py:13: RuntimeWarning: to-Python converter for boost::shared_ptr<caffe::Net<float> > already registered; second conversion method ignored.
  from ._caffe import Net, SGDSolver, NesterovSolver, AdaGradSolver, \
../../python/caffe/pycaffe.py:13: RuntimeWarning: to-Python converter for boost::shared_ptr<caffe::Blob<float> > already registered; second conversion method ignored.
  from ._caffe import Net, SGDSolver, NesterovSolver, AdaGradSolver, \
../../python/caffe/pycaffe.py:13: RuntimeWarning: to-Python converter for boost::shared_ptr<caffe::Solver<float> > already registered; second conversion method ignored.
  from ._caffe import Net, SGDSolver, NesterovSolver, AdaGradSolver, \


In [4]:
caffe.set_mode_cpu()
net = caffe.Net('../../models/swimmers_fullconv/swimmers_fullconv.prototxt', 
                '../../models/swimmers_fullconv/swimmers_fullconv_35x35_new.protobuf', caffe.TEST)
net.reshape()

In [5]:
# find appearances weight blob
appearance_weights = net.params['appearances'][0].data
print appearance_weights.shape

(13, 392, 1, 1)


In [6]:
B = np.where(appearance_weights[2]==1)[0]
print len(B), B

0 []


In [7]:
appearance_path = '/data/hdd/harzig/idpr/idpr_code/appearanceclasses.txt'

def read_appearances(file): 
    with open(appearance_path) as f:
        content = f.readlines()

    joints = {}

    for line in content:
        line = line.replace('\n', '')
        splitted = line.split("\t")
        joint = int(splitted[0])
        num = int(splitted[1])
        joints[joint] = map(int, splitted[2:2+num]) 

    return joints

In [8]:
joints = read_appearances(appearance_path)

for j in xrange(appearance_weights.shape[0]):
    for index in joints[j]:
        appearance_weights[j, index, 0, 0] = 1

In [9]:
B = np.where(appearance_weights[0]==1)[0]
print len(B), B

9 [  0  13  38  80  84 120 138 175 345]


In [10]:
idprclasses_path = '/data/hdd/harzig/idpr/idpr_code/idprclasses.txt'

def read_idprclasses(file): 
    with open(file) as f:
        content = f.readlines()

    classes = {}
    idx = 0
    #print content

    for line in content:
        line = line.replace('\n', '')
        splitted = line.split("\t")
        from_ = int(splitted[0])
        to_ = int(splitted[1])
        cluster_ = int(splitted[2])
        n_ = int(splitted[3])
        classes[idx] = map(int, splitted[4:4+n_]) 
        idx+=1
        #print idx

    return classes

In [11]:
classes = read_idprclasses(idprclasses_path)

In [12]:
print len(classes)

152


In [13]:
# find idprclasses weight blob
idprclasses_weights = net.params['idprclasses'][0].data
print idprclasses_weights.shape

(152, 392, 1, 1)


In [14]:
classes = read_idprclasses(idprclasses_path)
#print classes

for j in xrange(idprclasses_weights.shape[0]):
    for index in classes[j]:
        idprclasses_weights[j, index, 0, 0] = 1


In [15]:
B = np.where(idprclasses_weights[7]==1)[0]
print len(B), B

25 [ 14  49  71 115 130 134 139 174 176 207 213 218 239 244 256 292 306 316
 317 326 328 349 360 361 373]


In [16]:
# save changes to new binaryproto
net.save('../../models/swimmers_fullconv/swimmers_fullconv_35x35_new.protobuf')